In [2]:
import tensorflow as tf
import os
import pickle
import numpy as np
print(tf.__version__)
CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

1.13.1
['data_batch_1', 'data_batch_5', 'data_batch_2', 'test_batch', 'batches.meta', 'readme.html', 'data_batch_4', 'data_batch_3']


In [3]:
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]
train_filenames

['./cifar-10-batches-py/data_batch_1',
 './cifar-10-batches-py/data_batch_2',
 './cifar-10-batches-py/data_batch_3',
 './cifar-10-batches-py/data_batch_4',
 './cifar-10-batches-py/data_batch_5']

In [4]:
test_filenames 

['./cifar-10-batches-py/test_batch']

In [5]:
np.random.permutation(10)  # 随机排列序列

array([3, 2, 0, 5, 7, 8, 1, 6, 9, 4])

In [6]:
def load_data(filename):
    """返回值前面是图片特征，后面是图片分类标签"""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels'] 

    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)  # 垂直方向堆叠，变为ndarray
        self._data = self._data / 127.5 - 1  # 归一化，变为[-1,1]
        self._labels = np.hstack(all_labels)  # 水平方向堆叠，变为了ndarray
        print(self._data.shape)  # (50000, 3072)，因为每一张图像展平了
        print(self._labels.shape)
        print(self._labels[0:5])
        
        self._num_examples = self._data.shape[0]  # 拿样本数目
        self._need_shuffle = need_shuffle  # 进行随机
        self._indicator = 0  # 为了拿数据
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        """实现了shuffle"""
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)  # 生成随机下标
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data() 
                self._indicator = 0  # 回到开头
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:  # 余数不要，最后一次除不尽的数据不训练
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]  # 左闭右开，取特征
        batch_labels = self._labels[self._indicator: end_indicator]  # 取标签
        self._indicator = end_indicator
        return batch_data, batch_labels

    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
[6 9 9 4 1]
(10000, 3072)
(10000,)
[3 8 8 0 6]


In [7]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])  # 因为展平时把通道放在了前面，所以这里复原时也要把通道放在前面
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])  # 相当于轴滚动，与np.rollaxis功能一致的，变为(32,32,3)

# conv1: 神经元图， feature_map, 输出图像是None,32*32*32
conv1_1 = tf.layers.conv2d(x_image,
                           32,  # output channel number
                           (3,3),  # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv1_1')
conv1_2 = tf.layers.conv2d(conv1_1,
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv1_2')
# 16*16*32
pooling1 = tf.layers.max_pooling2d(conv1_2,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name = 'pool1')
conv2_1 = tf.layers.conv2d(pooling1,
                           32,  # output channel number
                           (3,3),  # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv2_1')
conv2_2 = tf.layers.conv2d(conv2_1,
                           32,  # output channel number
                           (3,3),  # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv2_2')
# 8*8*32
pooling2 = tf.layers.max_pooling2d(conv2_2,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name = 'pool2')
conv3_1 = tf.layers.conv2d(pooling2,
                           32,  # output channel number
                           (3,3),  # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv3_1')
conv3_2 = tf.layers.conv2d(conv3_1,
                           32,  # output channel number
                           (3,3),  # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv3_2')
# 4*4*32
pooling3 = tf.layers.max_pooling2d(conv3_2,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name = 'pool3')
# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)  # 先对logits计算softmax，再算交叉熵损失
#  y_ -> sofmax
#  y -> one_hot
#  loss = ylogy_

# 通过logits(也就是y_)来计算准确率
# indices
# [0，-2.2, 3.4, 6.8....]
predict = tf.argmax(y_, 1)

correct_prediction = tf.equal(predict, y)  # y是真实值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))  # tf.cast(x, dtype, name=None) 数据类型转换

with tf.name_scope('train_op'):  
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)  # 这里计算梯度并更新了梯度

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [8]:
import time

In [9]:
init = tf.global_variables_initializer()  # tf1.0必须做的
batch_size = 20
train_steps = 10000
test_steps = 100

# train 10k: 73.4%  训练一万次后的效果
with tf.Session() as sess:
    sess.run(init)  # 1.0必须做的
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(  # 1.0必须用sess.run
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 100 == 0:  # 训练100步，打印损失和准确率
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)  # 这一步是为了重新拿测试集
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)  # 拿测试集数据
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('now time=%f' % time.time())
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))          

[Train] Step: 100, loss: 1.97853, acc: 0.40000
[Train] Step: 200, loss: 2.04385, acc: 0.35000
[Train] Step: 300, loss: 1.55878, acc: 0.45000
[Train] Step: 400, loss: 1.53067, acc: 0.50000
[Train] Step: 500, loss: 1.39880, acc: 0.35000
[Train] Step: 600, loss: 1.55815, acc: 0.30000
[Train] Step: 700, loss: 1.34677, acc: 0.55000
[Train] Step: 800, loss: 1.69650, acc: 0.35000
[Train] Step: 900, loss: 1.77696, acc: 0.35000
[Train] Step: 1000, loss: 1.52550, acc: 0.40000
(10000, 3072)
(10000,)
[3 8 8 0 6]
now time=1658980417.963296
[Test ] Step: 1000, acc: 0.48450
[Train] Step: 1100, loss: 1.58141, acc: 0.45000
[Train] Step: 1200, loss: 1.13163, acc: 0.55000
[Train] Step: 1300, loss: 1.68525, acc: 0.35000
[Train] Step: 1400, loss: 1.24537, acc: 0.55000
[Train] Step: 1500, loss: 1.13023, acc: 0.50000
[Train] Step: 1600, loss: 1.27538, acc: 0.50000
[Train] Step: 1700, loss: 1.12575, acc: 0.65000
[Train] Step: 1800, loss: 0.91710, acc: 0.70000
[Train] Step: 1900, loss: 1.15205, acc: 0.55000
[T